In [49]:
import re
import pandas as pd

In [5]:
df = pd.read_csv("../data/Brite_list.csv")
df.head()

,0
0,NaN
1,Enzymes [BR:ko01000]
2,Transcription factors [BR:ko03000]
3,Secretion system [BR:ko02044]
4,G protein-coupled receptors [BR:ko04030]


In [12]:
BRITES = list(df.iloc[:,0])

In [13]:
BRITES.pop(0)

nan

In [14]:
BRITES

['Enzymes [BR:ko01000]',
 'Transcription factors [BR:ko03000]',
 'Secretion system [BR:ko02044]',
 'G protein-coupled receptors [BR:ko04030]',
 'GTP-binding proteins [BR:ko04031]',
 'Transporters [BR:ko02000]',
 'Chaperones and folding catalysts [BR:ko03110]',
 'KEGG modules [BR:ko00002]',
 'Ribosome biogenesis [BR:ko03009]',
 'Transfer RNA biogenesis [BR:ko03016]',
 'Polyketide biosynthesis proteins [BR:ko01008]',
 'Lectins [BR:ko04091]',
 'Spliceosome [BR:ko03041]',
 'Ubiquitin system [BR:ko04121]',
 'Proteoglycans [BR:ko00535]',
 'Prokaryotic defense system [BR:ko02048]',
 'Cell adhesion molecules and their ligands [BR:ko04516]',
 'Messenger RNA biogenesis [BR:ko03019]',
 'Exosome [BR:ko04147]',
 'Chromosome [BR:ko03036]',
 'Lipopolysaccharide biosynthesis proteins [BR:ko01005]',
 'Ion channels [BR:ko04040]',
 'Transcription machinery [BR:ko03021]',
 'Protein phosphatases and associated proteins [BR:ko01009]',
 'Cytoskeleton proteins [BR:ko04812]',
 'Bacterial motility proteins [BR:

In [17]:
BRITES[0].split("BR:")[1][:-1]

'ko01000'

In [18]:
brite_ids = [BRITE.split("BR:")[1][:-1] for BRITE in BRITES]

In [19]:
brite_ids

['ko01000',
 'ko03000',
 'ko02044',
 'ko04030',
 'ko04031',
 'ko02000',
 'ko03110',
 'ko00002',
 'ko03009',
 'ko03016',
 'ko01008',
 'ko04091',
 'ko03041',
 'ko04121',
 'ko00535',
 'ko02048',
 'ko04516',
 'ko03019',
 'ko04147',
 'ko03036',
 'ko01005',
 'ko04040',
 'ko03021',
 'ko01009',
 'ko04812',
 'ko02035',
 'ko01004',
 'ko03400',
 'ko03032',
 'ko04090',
 'ko03051',
 'ko01002',
 'ko02022',
 'ko03012',
 'ko03029',
 'ko04131',
 'ko02042',
 'ko00199',
 'ko01003',
 'ko01007',
 'ko04052',
 'ko04050',
 'ko00194',
 'ko01001',
 'ko00536']

In [1]:
f = open("../data/ko00001.keg", 'rU')

In [2]:
lines = []
for line in f:
    lines.append(line)

In [3]:
lines

['+D\tKO\n',
 '#<h2><a href="/kegg/kegg2.html"><img src="/Fig/bget/kegg3.gif" align="middle" border=0></a> &nbsp; KEGG Orthology (KO)</h2>\n',
 '#<!---\n',
 '#ENTRY       ko00001\n',
 '#NAME        KO\n',
 '#DEFINITION  KEGG Orthology (KO)\n',
 '#--->\n',
 '!\n',
 'A<b>Metabolism</b>\n',
 'B\n',
 'B  <b>Overview</b>\n',
 'C    01200 Carbon metabolism [PATH:ko01200]\n',
 'D      K00844  HK; hexokinase [EC:2.7.1.1]\n',
 'D      K12407  GCK; glucokinase [EC:2.7.1.2]\n',
 'D      K00845  glk; glucokinase [EC:2.7.1.2]\n',
 'D      K00886  ppgK; polyphosphate glucokinase [EC:2.7.1.63]\n',
 'D      K08074  ADPGK; ADP-dependent glucokinase [EC:2.7.1.147]\n',
 'D      K00918  pfkC; ADP-dependent phosphofructokinase/glucokinase [EC:2.7.1.146 2.7.1.147]\n',
 'D      K01810  GPI, pgi; glucose-6-phosphate isomerase [EC:5.3.1.9]\n',
 'D      K06859  pgi1; glucose-6-phosphate isomerase, archaeal [EC:5.3.1.9]\n',
 'D      K15916  pgi-pmi; glucose/mannose-6-phosphate isomerase [EC:5.3.1.9 5.3.1.8]\n',


In [63]:
level_1 = []
level_2 = []
brite_level_1 = {}
brite_level_2 = {}
for brite_id in brite_ids:
    for i, line in enumerate(lines):
        if 'A<b>' in line:
            level_1 = line
        if 'B  <b>' in line:
            level_2 = line
        if brite_id in line:
            level_1_match = re.search(u'<b>.*</b>', level_1).group()
            brite_level_1[brite_id] = level_1_match[3:-4]
            level_2_match = re.search(u'<b>.*</b>', level_2).group()
            brite_level_2[brite_id] = level_2_match[3:-4]

In [64]:
brite_level_1_df = pd.DataFrame(brite_level_1.items())
brite_level_1_df.columns = ['BRITE_id', 'level_1']
brite_level_1_df.head()

,BRITE_id,level_1
0,ko03029,Genetic Information Processing
1,ko00199,Metabolism
2,ko04050,Environmental Information Processing
3,ko00194,Metabolism
4,ko04121,Genetic Information Processing


In [65]:
brite_level_2_df = pd.DataFrame(brite_level_2.items())
brite_level_2_df.columns = ['BRITE_id', 'level_2']
brite_level_2_df.head()

,BRITE_id,level_2
0,ko03029,Translation
1,ko00199,Enzyme families
2,ko04050,Signaling molecules and interaction
3,ko00194,Energy metabolism
4,ko04121,"Folding, sorting and degradation"


In [66]:
result = pd.merge(brite_level_1_df,brite_level_2_df, on='BRITE_id')

In [67]:
result.head()

,BRITE_id,level_1,level_2
0,ko03029,Genetic Information Processing,Translation
1,ko00199,Metabolism,Enzyme families
2,ko04050,Environmental Information Processing,Signaling molecules and interaction
3,ko00194,Metabolism,Energy metabolism
4,ko04121,Genetic Information Processing,"Folding, sorting and degradation"


In [68]:
result

,BRITE_id,level_1,level_2
0,ko03029,Genetic Information Processing,Translation
1,ko00199,Metabolism,Enzyme families
2,ko04050,Environmental Information Processing,Signaling molecules and interaction
3,ko00194,Metabolism,Energy metabolism
4,ko04121,Genetic Information Processing,"Folding, sorting and degradation"
5,ko04040,Environmental Information Processing,Signaling molecules and interaction
6,ko02000,Environmental Information Processing,Membrane transport
7,ko04147,Cellular Processes,Transport and catabolism
8,ko02022,Environmental Information Processing,Signal transduction
9,ko03009,Genetic Information Processing,Translation


In [70]:
result.to_csv("../data/BRITE_key_levels.csv", index = False)